# Pandas Indexing

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'A': list(range(4)), 'B': list(range(4, 8))})
df

,A,B
0,0,4
1,1,5
2,2,6
3,3,7


We can swap the axes as follows:

In [3]:
df[['A', 'B']] = df[['B', 'A']]
df

,A,B
0,4,0
1,5,1
2,6,2
3,7,3


This, however, does not work:

In [4]:
df.loc[:, ['A', 'B']] = df[['B', 'A']]
df

,A,B
0,4,0
1,5,1
2,6,2
3,7,3


nor does this:

In [5]:
df.iloc[:, [0, 1]] = df.iloc[:, [1, 0]]
df

,A,B
0,4,0
1,5,1
2,6,2
3,7,3


The reason is that when we use `.loc` or `.iloc` pandas automatically *aligns* the axes.

## Using `.loc`

The `.loc` attribute is the primary access method. It accepts:

- Single labels.
- List or array of labels.
- Slices of labels `'a':'f'`, `'c':` etc.

A case like `'c':` will work only when the index is sorted. If not, a `KeyError` is returned.

### With Series

In [6]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))
s1.loc[:'c']

a   -1.172907
b   -0.524277
c   -0.938370
dtype: float64

### With Data Frames

With data frames we need to be explicit when we want to index/slice the columns.

In [7]:
df1 = pd.DataFrame(np.random.randn(6, 4), index=list('abcdef'),
columns=list('ABCD'))
df1

,A,B,C,D
a,0.506867,-0.129144,-2.325755,-0.765544
b,-0.695209,-0.962873,0.257549,-0.421503
c,2.166315,1.183305,0.418292,1.127155
d,-2.621619,0.060626,-0.556854,-0.311262
e,-0.306600,0.919141,0.060670,1.444010
f,0.786331,-0.343798,-0.281035,0.782649


In [8]:
df1.loc['b':'d']

,A,B,C,D
b,-0.695209,-0.962873,0.257549,-0.421503
c,2.166315,1.183305,0.418292,1.127155
d,-2.621619,0.060626,-0.556854,-0.311262


In [9]:
df1.loc[:, ['A', 'B']]

,A,B
a,0.506867,-0.129144
b,-0.695209,-0.962873
c,2.166315,1.183305
d,-2.621619,0.060626
e,-0.306600,0.919141
f,0.786331,-0.343798


Same reasoning with booleans.

In [10]:
df1.loc['a'] > 0

A     True
B    False
C    False
D    False
Name: a, dtype: bool

In [11]:
df1.loc['a':'c'] < 0

,A,B,C,D
a,False,True,True,True
b,True,True,False,True
c,False,False,False,False


In [12]:
np.where(df1.loc['a':'c'] < 0)

(array([0, 0, 0, 1, 1, 1]), array([1, 2, 3, 0, 1, 3]))

### Indexing with missing values

Indexing with a list containing missing values is now deprecated.

In [13]:
df1.loc[['a', np.nan, 'c']]

/home/giovenko/miniconda3/envs/sklearn/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,A,B,C,D
a,0.506867,-0.129144,-2.325755,-0.765544
NaN,NaN,NaN,NaN,NaN
c,2.166315,1.183305,0.418292,1.127155


Similarly, indexing with a non-matching index produce the same message.

In [14]:
df1.loc[['a', 'c', 'x']]

/home/giovenko/miniconda3/envs/sklearn/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,A,B,C,D
a,0.506867,-0.129144,-2.325755,-0.765544
c,2.166315,1.183305,0.418292,1.127155
x,NaN,NaN,NaN,NaN


The recommended way is to use `reindex`.

In [15]:
df1.reindex(['a', 'c', 'x'])

,A,B,C,D
a,0.506867,-0.129144,-2.325755,-0.765544
c,2.166315,1.183305,0.418292,1.127155
x,NaN,NaN,NaN,NaN


To avoid including `NaN`s, select only valid indexes taking advantage of the equivalence between indexes and sets.

In [16]:
df1.loc[df1.index.intersection(['a', 'c', 'x'])]

,A,B,C,D
a,0.506867,-0.129144,-2.325755,-0.765544
c,2.166315,1.183305,0.418292,1.127155


## Using `.iloc`

As before, we need to be explicit when indexing columns.

In [17]:
df2 = pd.DataFrame(np.random.randn(6, 4), index=list(range(0, 12, 2)),
columns=list(range(0, 8, 2)))
df2

,0,2,4,6
0,-1.174636,-0.350344,0.376640,-0.565246
2,0.640391,-1.405737,-0.124564,-0.243579
4,-0.657400,-1.299994,1.170997,-0.154710
6,-1.403326,-0.382572,0.060716,-0.930173
8,1.204765,-0.734977,1.959325,1.673952
10,-0.486738,0.762831,-0.850076,2.209039


Having a limit out of range is not a problem.

In [18]:
df2.iloc[:, 2:10]

,4,6
0,0.376640,-0.565246
2,-0.124564,-0.243579
4,1.170997,-0.154710
6,0.060716,-0.930173
8,1.959325,1.673952
10,-0.850076,2.209039


In [19]:
df2.iloc[0:100]

,0,2,4,6
0,-1.174636,-0.350344,0.376640,-0.565246
2,0.640391,-1.405737,-0.124564,-0.243579
4,-0.657400,-1.299994,1.170997,-0.154710
6,-1.403326,-0.382572,0.060716,-0.930173
8,1.204765,-0.734977,1.959325,1.673952
10,-0.486738,0.762831,-0.850076,2.209039


If all the indices are out of range, an empty data frame is returned.

In [20]:
df2.iloc[:, 4:5]

""
0
2
4
6
8
10


We can subset using lists.

In [21]:
df2.iloc[[1, 2, 1], [0, 2]]

,0,4
2,0.640391,-0.124564
4,-0.657400,1.170997
2,0.640391,-0.124564


`iloc` and `iat` equivalence.

In [22]:
df2.iloc[1, 1] == df2.iat[1, 1]

True

Equivalence, but not identity.

In [23]:
df2.iloc[1, 1] is df2.iat[1, 1]

False

In [24]:
np.random.seed(1024)
df = pd.DataFrame(np.random.randn(10, 3), columns=list('abc'))
df[(df.a < df.b) & (df.b < df.c)]

,a,b,c
3,-0.568507,-0.065101,0.806817
9,-0.425498,-0.402577,0.604104


In [25]:
df.query('(a < b) & (b < c)')

,a,b,c
3,-0.568507,-0.065101,0.806817
9,-0.425498,-0.402577,0.604104


## Renaming columns

The columns, or more in general the index (no matter which axis) of a data frame can be renamed via the `.rename()` method, which takes a mapping between the old and the new columns.

In [26]:
df2 = df.copy()
df2.columns = ['Bad Name', 'Ugly Name', 'Terrible Name']
df2

,Bad Name,Ugly Name,Terrible Name
0,2.124449,0.252646,1.454179
1,0.569240,0.458224,-0.809333
2,0.864073,0.201701,-1.875299
3,-0.568507,-0.065101,0.806817
4,-0.577818,0.573061,-0.336675
5,0.297007,-0.374804,0.155105
6,0.704857,0.845218,-0.658181
7,0.568106,0.515381,-0.615650
8,0.926114,-1.285913,1.430140
9,-0.425498,-0.402577,0.604104


Renaming can be done either via a mapping or with a function. The mapping can operate on individual entries. One can either specify the `axis` or pass the mapper or function to `index` or `columns`.

With a mapping.

In [27]:
df2.rename(mapper={'Bad Name': 'better_name'}, axis=1)

,better_name,Ugly Name,Terrible Name
0,2.124449,0.252646,1.454179
1,0.569240,0.458224,-0.809333
2,0.864073,0.201701,-1.875299
3,-0.568507,-0.065101,0.806817
4,-0.577818,0.573061,-0.336675
5,0.297007,-0.374804,0.155105
6,0.704857,0.845218,-0.658181
7,0.568106,0.515381,-0.615650
8,0.926114,-1.285913,1.430140
9,-0.425498,-0.402577,0.604104


In [28]:
df2.rename(columns={'Terrible Name': 'nice_name'})

,Bad Name,Ugly Name,nice_name
0,2.124449,0.252646,1.454179
1,0.569240,0.458224,-0.809333
2,0.864073,0.201701,-1.875299
3,-0.568507,-0.065101,0.806817
4,-0.577818,0.573061,-0.336675
5,0.297007,-0.374804,0.155105
6,0.704857,0.845218,-0.658181
7,0.568106,0.515381,-0.615650
8,0.926114,-1.285913,1.430140
9,-0.425498,-0.402577,0.604104


With a function.

In [29]:
df2.rename(columns=lambda x: x.lower().replace(' ', '_'))

,bad_name,ugly_name,terrible_name
0,2.124449,0.252646,1.454179
1,0.569240,0.458224,-0.809333
2,0.864073,0.201701,-1.875299
3,-0.568507,-0.065101,0.806817
4,-0.577818,0.573061,-0.336675
5,0.297007,-0.374804,0.155105
6,0.704857,0.845218,-0.658181
7,0.568106,0.515381,-0.615650
8,0.926114,-1.285913,1.430140
9,-0.425498,-0.402577,0.604104


## Equivalent of `case_when` with `where`

`where` applies an if-then-else condition. If the condition is true, the value is left unchanged, otherwise the `other` value is used.

In [30]:
df3 = pd.DataFrame(np.random.randint(low=1, high=11, size=24).reshape(6, 4),
index=list('abcdef'), columns = list('ABCD'))

df3

,A,B,C,D
a,9,8,7,5
b,6,6,4,1
c,6,10,10,4
d,6,2,1,6
e,6,8,3,2
f,8,10,7,10


In [31]:
df3.where(df3 > 5, 0)

,A,B,C,D
a,9,8,7,0
b,6,6,0,0
c,6,10,10,0
d,6,0,0,6
e,6,8,0,0
f,8,10,7,10


## Names and reset index

When using `reset_index` the index is moved to a column named `index`. If we name the column using `names`, this will become the new name of the column.

In [39]:
df4 = pd.DataFrame(np.random.randint(0, 11, 9).reshape(3, 3),
index=['a', 'b', 'c'], 
columns=['A', 'B', 'C'])

df4

,A,B,C
a,0,8,2
b,1,0,0
c,7,3,4


In [40]:
df4.reset_index()

,index,A,B,C
0,a,0,8,2
1,b,1,0,0
2,c,7,3,4


In [41]:
df4.index.name = 'labels'
df4.reset_index()

,labels,A,B,C
0,a,0,8,2
1,b,1,0,0
2,c,7,3,4
